In [184]:
#import standard libraries
import pandas as pd
import numpy as np

#import ML specific resources:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#Import data provided by Kaggle:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submision_example = pd.read_csv("gender_submission.csv")


train.describe(include='all')
# train.isna().sum()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Cumings, Mrs. John Bradley (Florence Briggs Th...",male,NaN,NaN,NaN,CA. 2343,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [89]:
# Step 1: Exploration and 
np.random.seed(26)

X = train.drop("Survived",axis=1)
Y = train['Survived']

X['Sex'] = np.where(X['Sex']=='male',1,0 ) 

X = X.drop(["Name","Age","Cabin"],axis=1)

min_max_scaler = preprocessing.MinMaxScaler()
X['Normalized_Fare'] = min_max_scaler.fit_transform(X[["Fare"]])

X = X.drop(["Ticket","Fare"],axis=1)
X = pd.get_dummies(X[["PassengerId","Pclass","Sex","SibSp","Parch","Embarked","Normalized_Fare"]])




PassengerId        0
Pclass             0
Sex                0
SibSp              0
Parch              0
Normalized_Fare    0
Embarked_C         0
Embarked_Q         0
Embarked_S         0
dtype: int64

In [90]:
X_train,X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)

In [158]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 100, max_features=0.8,oob_score =True,random_state=50)

# We'll keep the default hyperprarameters
clf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 0.8,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': True,
 'random_state': 50,
 'verbose': 0,
 'warm_start': False}

In [159]:
clf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=0.8, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=50, verbose=0,
                       warm_start=False)

In [160]:
# Now we can make a prediction by passing a np array
Y_preds = clf.predict(X_test)
Y_preds

array([1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [161]:
# 4. Evaluate the model:
clf.score(X_train, Y_train) # Returns the mean accuracy on the given test data and labels -- 1.0 is max 100%

1.0

In [162]:
clf.score(X_test,Y_test)

0.7988826815642458

In [163]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(Y_test,Y_preds)) 

              precision    recall  f1-score   support

           0       0.83      0.87      0.85       116
           1       0.74      0.67      0.70        63

    accuracy                           0.80       179
   macro avg       0.78      0.77      0.77       179
weighted avg       0.80      0.80      0.80       179



In [164]:
confusion_matrix(Y_test,Y_preds)

array([[101,  15],
       [ 21,  42]])

In [165]:
accuracy_score(Y_test,Y_preds)

0.7988826815642458

In [166]:
# for i in range(10,100,10):
#     print(f"Trying model with {i} estimatores:")
#     clf = RandomForestClassifier(n_estimators = i).fit(X_train,Y_train)
#     print(f"Model Accuracy on test set: {clf.score(X_test,Y_test)*100:.2f}")
    

# for i in range(1,100,10):
#     print(f"Trying model with {i} estimatores:")
#     clf = RandomForestClassifier(min_samples_leaf = i).fit(X_train,Y_train)
#     print(f"Model Accuracy on test set: {clf.score(X_test,Y_test)*100:.2f}")


Trying model with 10 estimatores:
Model Accuracy on test set: 73.74
Trying model with 20 estimatores:
Model Accuracy on test set: 76.54
Trying model with 30 estimatores:
Model Accuracy on test set: 75.42
Trying model with 40 estimatores:
Model Accuracy on test set: 75.42
Trying model with 50 estimatores:
Model Accuracy on test set: 76.54
Trying model with 60 estimatores:
Model Accuracy on test set: 75.42
Trying model with 70 estimatores:
Model Accuracy on test set: 78.77
Trying model with 80 estimatores:
Model Accuracy on test set: 74.86
Trying model with 90 estimatores:
Model Accuracy on test set: 78.21


In [185]:



test['Sex'] = np.where(test['Sex']=='male',1,0 ) 

test = test.drop(["Name","Age","Cabin"],axis=1)

min_max_scaler = preprocessing.MinMaxScaler()
test['Normalized_Fare'] = min_max_scaler.fit_transform(test[["Fare"]])

test = test.drop(["Ticket","Fare"],axis=1)
test = pd.get_dummies(test[["PassengerId","Pclass","Sex","SibSp","Parch","Embarked","Normalized_Fare"]])
test






,PassengerId,Pclass,Sex,SibSp,Parch,Normalized_Fare,Embarked_C,Embarked_Q,Embarked_S
0,892,3,1,0,0,0.015282,0,1,0
1,893,3,0,1,0,0.013663,0,0,1
2,894,2,1,0,0,0.018909,0,1,0
3,895,3,1,0,0,0.016908,0,0,1
4,896,3,0,1,1,0.023984,0,0,1
...,...,...,...,...,...,...,...,...,...
413,1305,3,1,0,0,0.015713,0,0,1
414,1306,1,0,0,0,0.212559,1,0,0
415,1307,3,1,0,0,0.014151,0,0,1
416,1308,3,1,0,0,0.015713,0,0,1


In [204]:
# test
# test.isna().sum()
test["Normalized_Fare"] = np.where(test['Normalized_Fare'].isna(),0,test['Normalized_Fare'] )

predictions = clf.predict(test)

predictions = pd.DataFrame(predictions)
predictions

ids = test[['PassengerId']]
ids

titanic_1 = pd.DataFrame(pd.concat([ids,predictions], axis=1))

titanic_1 = titanic_1.rename(columns={titanic_1.columns[1]:"Survived"})
titanic_1


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [205]:
titanic_1.to_csv("/Users/kurtis.campbell/Desktop/Kurtis/Kaggle/Titanic/predictions.csv")